In [2]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching dispersion_gauss_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s dispersion_gauss_Scene"

Jupyter Capture Output v0.0.11


In [212]:
sigma_x = 1
k_0 = 2.5
c = 4


# +++ dispersion relations +++

m_1 = 100
m_2 = 1

# no dispersion
def omega(k):
    return c * k

# dispersion 1
def omega_1(k):
    return np.sqrt(m_1 * k)

# dispersion 2
def omega_2(k):
    return np.sqrt(m_2 * k**3)



# +++ no dispersion +++

# real space
def get_Psi(t):
    def Psi(x):
        return np.exp(-(x-c*t)**2 / (2*sigma_x**2)) * np.cos(k_0 * (x-c*t))
    return Psi

def get_Psi_envelope(t, sign = 1):
    def Psi_envelope(x):
        return sign * np.exp(-(x-c*t)**2 / (2*sigma_x**2))
    return Psi_envelope


# fourier space
def get_Psi_hat(t):
    def Psi_hat(k):
        return sigma_x * np.exp(-sigma_x**2/2 * (k-k_0)**2) * np.cos(-omega(k) * t)
    return Psi_hat

def get_Psi_hat_envelope(t, sign = 1):
    def Psi_envelope(k):
        return sign * sigma_x * np.exp(-sigma_x**2/2 * (k-k_0)**2)
    return Psi_envelope



# +++ dispersion +++
omega_disp = omega_1

omega_0 = omega_disp(k_0)
v_gr = np.sqrt(m_1 / (2*k_0))
omega_dd_0 = -np.sqrt(m_1 / k_0**3) / 8
def alpha(t):
    return omega_dd_0 * t / sigma_x**2


# real space
def get_Psi_dispersion(t):
	def Psi(x):
		return np.cos(k_0*x - omega_0*t) * np.real(np.exp(-(x-v_gr*t)**2 / (2*sigma_x**2*np.sqrt(1 + 1j*alpha(t)**2))) / np.sqrt(1 + 1j*alpha(t)**2))
	return Psi

def get_Psi_dispersion_envelope(t, sign = 1):
    def Psi_envelope(x):
        return sign * np.real(np.exp(-(x-v_gr*t)**2 / (2*sigma_x**2*np.sqrt(1 + 1j*alpha(t)**2))) / np.sqrt(1 + 1j*alpha(t)**2))
    return Psi_envelope


# fourier space
def get_Psi_dispersion_hat(t):
    def Psi_hat(k):
        return sigma_x * np.exp(-sigma_x**2/2 * (k-k_0)**2) * np.cos(-omega_disp(k) * t)
    return Psi_hat

def get_Psi_hat_dispersion_envelope(t, sign = 1):
    def Psi_envelope(k):
        return sign * sigma_x * np.exp(-sigma_x**2/2 * (k-k_0)**2)
    return Psi_envelope

In [217]:
%%manim -qh --fps 60 $video_scene

class dispersion_gauss_Scene(Scene):
	def construct(self):
		CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
		self.add(CVC)

		# headline
		headline = Title(r"Gaußsches Wellenpaket ohne und mit Dispersion", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
		headline[0][20:24].set_color(RED)
		headline[0][27:30].set_color(BLUE)
		self.add(headline)

		ax_pos = {
			"no_disp_real_space": [-3, 1, 0],
			"no_disp_fourier_space": [-3, -2, 0],
			"disp_real_space": [3, 1, 0],
			"disp_fourier_space": [3, -2, 0],
		}

		# ax_pos = {
		# 	"no_disp_real_space": [-3, -0.5, 0],
		# 	"no_disp_fourier_space": [-3, -2, 0],
		# 	"disp_real_space": [3, -0.5, 0],
		# 	"disp_fourier_space": [3, -2, 0],
		# }

		# time parameters
		x_max = 25
		t0 = 0
		t1 = 5

		# create ax
		x_length = 4.5
		y_length = 2
		x_range = [0, x_max, 1]
		y_range = [-1, 1.25, 2]
		plot_range = [0, 0.95*x_max]


		# no dispersion: real space
		ax1 = Axes(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, 
			tips = True, axis_config = {"include_ticks": False, "tip_width": 0.15, "tip_height": 0.15}).move_to(ax_pos["no_disp_real_space"]).set_opacity(0.5)
		ax1_xlabel = ax1.get_x_axis_label(Tex(r"$x$", font_size = 36, color = WHITE))
		ax1_ylabel = ax1.get_y_axis_label(Tex(r"$\psi(x, t)$", font_size = 36, color = WHITE))
		self.add(ax1, ax1_xlabel, ax1_ylabel)

		psi1 = ax1.plot(get_Psi(t0), color = RED, stroke_width = 2, x_range = plot_range)
		psi1.ax = ax1
		psi1.color = RED
		psi1.psi_getter = get_Psi
		psi1.envelope_getter = get_Psi_envelope
		psi1.envelope_top = ax1.plot(psi1.envelope_getter(t0), color = WHITE, stroke_width = 2, x_range = plot_range)
		psi1.envelope_down = ax1.plot(psi1.envelope_getter(t0, sign = -1), color = WHITE, stroke_width = 2, x_range = plot_range)
		self.add(psi1, psi1.envelope_top, psi1.envelope_down)


		# no dispersion: fourier space
		ax2 = Axes(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, 
			tips = True, axis_config = {"include_ticks": False, "tip_width": 0.15, "tip_height": 0.15}).move_to(ax_pos["no_disp_fourier_space"])
		ax2_xlabel = ax2.get_x_axis_label(Tex(r"$k$", font_size = 36, color = WHITE))
		ax2_ylabel = ax2.get_y_axis_label(Tex(r"$\hat{\psi}(k, t)$", font_size = 36, color = WHITE))
		self.add(ax2, ax2_xlabel, ax2_ylabel)

		psi2 = ax2.plot(get_Psi_hat(t0), color = RED, stroke_width = 2, x_range = plot_range)
		psi2.ax = ax2
		psi2.color = RED
		psi2.psi_getter = get_Psi_hat
		psi2.envelope_getter = get_Psi_hat_envelope
		psi2.envelope_top = ax2.plot(psi2.envelope_getter(t0), color = WHITE, stroke_width = 2, x_range = plot_range)
		psi2.envelope_down = ax2.plot(psi2.envelope_getter(t0, sign = -1), color = WHITE, stroke_width = 2, x_range = plot_range)
		self.add(psi2, psi2.envelope_top, psi2.envelope_down)

		# dispersion: real space
		ax3 = Axes(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, 
			tips = True, axis_config = {"include_ticks": False, "tip_width": 0.15, "tip_height": 0.15}).move_to(ax_pos["disp_real_space"])
		ax3_xlabel = ax3.get_x_axis_label(Tex(r"$x$", font_size = 36, color = WHITE))
		ax3_ylabel = ax3.get_y_axis_label(Tex(r"$\psi(x, t)$", font_size = 36, color = WHITE))
		self.add(ax3, ax3_xlabel, ax3_ylabel)

		psi3 = ax3.plot(get_Psi_dispersion(t0), color = BLUE, stroke_width = 2, x_range = plot_range)
		psi3.ax = ax3
		psi3.color = BLUE
		psi3.psi_getter = get_Psi_dispersion
		psi3.envelope_getter = get_Psi_dispersion_envelope
		psi3.envelope_top = ax3.plot(psi3.envelope_getter(t0), color = WHITE, stroke_width = 2, x_range = plot_range)
		psi3.envelope_down = ax3.plot(psi3.envelope_getter(t0, sign = -1), color = WHITE, stroke_width = 2, x_range = plot_range)
		self.add(psi3, psi3.envelope_top, psi3.envelope_down)

		# dispersion: fourier space
		ax4 = Axes(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, 
			tips = True, axis_config = {"include_ticks": False, "tip_width": 0.15, "tip_height": 0.15}).move_to(ax_pos["disp_fourier_space"])
		ax4_xlabel = ax4.get_x_axis_label(Tex(r"$k$", font_size = 36, color = WHITE))
		ax4_ylabel = ax4.get_y_axis_label(Tex(r"$\hat{\psi}(k, t)$", font_size = 36, color = WHITE))
		self.add(ax4, ax4_xlabel, ax4_ylabel)

		psi4 = ax4.plot(get_Psi_dispersion_hat(t0), color = BLUE, stroke_width = 2, x_range = plot_range)
		psi4.ax = ax4
		psi4.color = BLUE
		psi4.psi_getter = get_Psi_dispersion_hat
		psi4.envelope_getter = get_Psi_hat_dispersion_envelope
		psi4.envelope_top = ax4.plot(psi4.envelope_getter(t0), color = WHITE, stroke_width = 2, x_range = plot_range)
		psi4.envelope_down = ax4.plot(psi4.envelope_getter(t0, sign = -1), color = WHITE, stroke_width = 2, x_range = plot_range)
		self.add(psi4, psi4.envelope_top, psi4.envelope_down)


		def psi_updater(psi):
			t = t_tracker.get_value()
			psi_ax = psi.ax
			psi_color = psi.color
			psi_psi_getter = psi.psi_getter
			psi_envelope_getter = psi.envelope_getter
			psi.become(psi.ax.plot(psi_psi_getter(t), color = psi_color, stroke_width = 2, x_range = plot_range))
			if psi_envelope_getter:
				psi.envelope_top.become(psi.ax.plot(psi_envelope_getter(t), color = WHITE, stroke_width = 2, x_range = plot_range))
				psi.envelope_down.become(psi.ax.plot(psi_envelope_getter(t, sign = -1), color = WHITE, stroke_width = 2, x_range = plot_range))
		

		# animation
		self.wait(1)
		t_tracker = ValueTracker(t0)
		psi1.add_updater(psi_updater)
		psi2.add_updater(psi_updater)
		psi3.add_updater(psi_updater)
		psi4.add_updater(psi_updater)
		self.play(t_tracker.animate.set_value(t1), rate_func = linear, run_time = 4*(t1-t0))
		self.wait(3)


Manim Community v0.18.1